# Tutorial 1: Basics on Compression and the blosc2.NDArray Object

In [1]:
import blosc2
import numpy as np

## Basic compress2/decompress2

In [2]:
# Let's create a simple tensor
a = np.arange(1000_000).reshape(1000, 1000)

In [3]:
compressed_a = blosc2.compress2(a)
cbytes = len(compressed_a)
nbytes = a.size * a.itemsize
print(f"cbytes: {cbytes}, cratio: {nbytes / cbytes:.2f}x")

cbytes: 12732, cratio: 628.34x


In [4]:
a2 = blosc2.decompress2(compressed_a)
print(f"{type(a2)=}, {len(a2)=}")

type(a2)=<class 'bytes'>, len(a2)=8000000


In [5]:
# Restore (requires effort and metainfo supplement)
a3 = np.frombuffer(a2, dtype=np.int64).reshape(1000, 1000)
a3

array([[     0,      1,      2, ...,    997,    998,    999],
       [  1000,   1001,   1002, ...,   1997,   1998,   1999],
       [  2000,   2001,   2002, ...,   2997,   2998,   2999],
       ...,
       [997000, 997001, 997002, ..., 997997, 997998, 997999],
       [998000, 998001, 998002, ..., 998997, 998998, 998999],
       [999000, 999001, 999002, ..., 999997, 999998, 999999]],
      shape=(1000, 1000))

### More fancy compression for tensors

In [6]:
compressed_t = blosc2.pack_tensor(a)
cbytes = len(compressed_t)
print(f"cbytes: {cbytes}, cratio: {nbytes / cbytes:.2f}x")

cbytes: 13000, cratio: 615.38x


In [7]:
# Restore (fancier)
a4 = blosc2.unpack_tensor(compressed_t)
print(f"{type(a4)=}, {len(a4)=}")
a4

type(a4)=<class 'numpy.ndarray'>, len(a4)=1000


array([[     0,      1,      2, ...,    997,    998,    999],
       [  1000,   1001,   1002, ...,   1997,   1998,   1999],
       [  2000,   2001,   2002, ...,   2997,   2998,   2999],
       ...,
       [997000, 997001, 997002, ..., 997997, 997998, 997999],
       [998000, 998001, 998002, ..., 998997, 998998, 998999],
       [999000, 999001, 999002, ..., 999997, 999998, 999999]],
      shape=(1000, 1000))

## Compression params
Let's see how to copy the NDArray data whilst altering the compression parameters. This may be useful in many contexts, for example testing how changing the codec of an existing array affects the compression ratio.

In [8]:
cparams = blosc2.CParams(
    codec=blosc2.Codec.LZ4,
    clevel=9,
    filters=[blosc2.Filter.SHUFFLE, blosc2.Filter.BYTEDELTA],
)

compressed_t = blosc2.pack_tensor(a, cparams=cparams)
cbytes = len(compressed_t)
print(f"cbytes: {cbytes}, cratio: {nbytes / cbytes:.2f}x")

cbytes: 21569, cratio: 370.90x


In this case the compression ratio is quite lower than before, since we have changed to a different codec that is optimised for compression speed, not compression ratio. In general there is a tradeoff between the two.

### Native Blosc2 codecs
Blosc2 supports many standard codecs, since there is no one-size-fits-all compression solution - one codec may be perfect for one context, but quite suboptimal in another.
* ZLIB codec: uses the DEFLATE algorithm, is standard, and works well for images.
* ZSTD codec: better compression ratio to ZLIB and faster compression/decompression (the default in Blosc2).
* LZ4 codec: faster comp/decomp than ZSTD but reduced compression ratio.
* BloscLZ: new implementation of the simple FASTLZ algorithm; similar tradeoff to LZ4, but the latter is generally better.


**Exercise**: Use different codecs (BLOSCLZ, LZ4, LZ4HC, ZLIB and ZSTD), clevels (0-9) and filters (SHUFFLE, BITSHUFFLE, BYTEDELTA) above and see how cratios vary.

Hint: the BYTEDELTA filter is meant to be applied after a SHUFFLE filter; otherwise, the results will be sub-optimal.

In [9]:
# SOLUTION
cparams = blosc2.CParams(
    codec=blosc2.Codec.ZSTD,
    clevel=9,
    filters=[blosc2.Filter.SHUFFLE, blosc2.Filter.BYTEDELTA],
)

compressed_t = blosc2.pack_tensor(a, cparams=cparams)
cbytes = len(compressed_t)
print(f"cbytes: {cbytes}, cratio: {nbytes / cbytes:.2f}x")

cbytes: 982, cratio: 8146.64x


Finally, via package extensions to Blosc2, one may access the JPEG2000 family of compression algorithms, which aim for a compromise between compression ratio and image quality; Blosc2 implements plugins for [GROK](https://github.com/Blosc/blosc2_grok) and [OPENHTJ2K](https://github.com/Blosc/blosc2_openhtj2k) for a convenient way to access JPEG2000 compression.

# NDArray: A NDim, Compressed Data Container

NDArray objects let users perform different operations with  arrays like setting, copying or slicing them. In this section, we are going to see how to create and manipulate these NDArray arrays, which possess metadata and data. The data is *chunked* and *compressed*; the metadata gives information about the data itself, as well as the chunking and compression. Chunking and compression are features which make NDArray arrays very efficient for working with large data.

## Creating an array
Let's start by creating a 2D array with 100M elements filled with ``arange``. We can then print out the metadata, which contains information about: the array data (such as ``shape`` and ``dtype``); and how the data is compressed and stored, such as chunk- and block-shapes (``chunks`` and ``blocks``) and compression params (``CParams``). See [here](https://www.blosc.org/python-blosc2/getting_started/overview.html) for an explanation of chunking and blocking.



In [10]:
shape = (10_000, 10_000)
array = blosc2.arange(np.prod(shape), shape=shape, dtype=np.int32)
print(array.info)

type    : NDArray
shape   : (10000, 10000)
chunks  : (1250, 10000)
blocks  : (10, 10000)
dtype   : int32
nbytes  : 400000000
cbytes  : 1093039
cratio  : 365.95
cparams : CParams(codec=<Codec.ZSTD: 5>, codec_meta=0, clevel=5, use_dict=False, typesize=4,
        : nthreads=8, blocksize=400000, splitmode=<SplitMode.AUTO_SPLIT: 3>,
        : filters=[<Filter.NOFILTER: 0>, <Filter.NOFILTER: 0>, <Filter.NOFILTER: 0>,
        : <Filter.NOFILTER: 0>, <Filter.NOFILTER: 0>, <Filter.SHUFFLE: 1>], filters_meta=[0, 0,
        : 0, 0, 0, 0], tuner=<Tuner.STUNE: 0>)
dparams : DParams(nthreads=8)



The ``cratio`` parameter tells us how effective the compression is, since it gives the ratio between the number of bytes required to store the array in uncompressed and compressed form. Here we require almost 300x less space for the compressed array! Note that all the compression and decompression parameters are set to the default, and ``chunks`` and ``blocks`` have been selected automatically - playing around with them will affect the ``cratio`` (as well as compression and decompression speed).

We can also create an NDArray by compressing a NumPy array:

In [11]:
nparray = np.linspace(0, 100, np.prod(shape), dtype=np.float64).reshape(shape)
b2array = blosc2.asarray(nparray)
print(b2array.info)

type    : NDArray
shape   : (10000, 10000)
chunks  : (625, 10000)
blocks  : (5, 10000)
dtype   : float64
nbytes  : 800000000
cbytes  : 14833410
cratio  : 53.93
cparams : CParams(codec=<Codec.ZSTD: 5>, codec_meta=0, clevel=5, use_dict=False, typesize=8,
        : nthreads=8, blocksize=400000, splitmode=<SplitMode.AUTO_SPLIT: 3>,
        : filters=[<Filter.NOFILTER: 0>, <Filter.NOFILTER: 0>, <Filter.NOFILTER: 0>,
        : <Filter.NOFILTER: 0>, <Filter.NOFILTER: 0>, <Filter.SHUFFLE: 1>], filters_meta=[0, 0,
        : 0, 0, 0, 0], tuner=<Tuner.STUNE: 0>)
dparams : DParams(nthreads=8)



or an iterator:

In [12]:
N = 1000_000
rng = np.random.default_rng()
it = ((-x + 1, x - 2, rng.normal()) for x in range(N))
sa = blosc2.fromiter(it, dtype="i4,f4,f8", shape=(N,))
print(sa.info)
print(f"first 3 rows of sa: {sa[:3]}")

type    : NDArray
shape   : (1000000,)
chunks  : (1000000,)
blocks  : (62500,)
dtype   : [('f0', '<i4'), ('f1', '<f4'), ('f2', '<f8')]
nbytes  : 16000000
cbytes  : 7060260
cratio  : 2.27
cparams : CParams(codec=<Codec.ZSTD: 5>, codec_meta=0, clevel=5, use_dict=False, typesize=16,
        : nthreads=8, blocksize=1000000, splitmode=<SplitMode.AUTO_SPLIT: 3>,
        : filters=[<Filter.NOFILTER: 0>, <Filter.NOFILTER: 0>, <Filter.NOFILTER: 0>,
        : <Filter.NOFILTER: 0>, <Filter.NOFILTER: 0>, <Filter.SHUFFLE: 1>], filters_meta=[0, 0,
        : 0, 0, 0, 0], tuner=<Tuner.STUNE: 0>)
dparams : DParams(nthreads=8)

first 3 rows of sa: [( 1, -2., 0.71085376) ( 0, -1., 0.1804048 ) (-1,  0., 0.48975023)]


**Exercise:** Create a 2D NDArray with shape (1000, 10_000) and filled with sequential integers using the `range` iterator. Then use `blosc2.arange` to create the same array, and check that the two arrays are equal.  Use `%time` magick tool to time the two operations.  What do you notice about the time taken?  Why do you think this is happening?

In [13]:
# SOLUTION
%time array_from_iter = blosc2.fromiter(range(1000 * 10_000), dtype='i4', shape=(1000, 10_000))
%time array_from_arange = blosc2.arange(10_000_000, shape=(1000, 10_000))
# For reference
%time array_from_arange = np.arange(10_000_000).reshape((1000, 10_000))

CPU times: user 1.16 s, sys: 88.3 ms, total: 1.25 s
Wall time: 1.1 s
CPU times: user 129 ms, sys: 18 ms, total: 147 ms
Wall time: 89 ms
CPU times: user 3.61 ms, sys: 2.5 ms, total: 6.11 ms
Wall time: 6.12 ms



## Reading and modifying data
NDArray arrays cannot be read directly, since they are compressed, and so must be decompressed first (to NumPy arrays, which are stored in memory). This can be done for the full array using the ``[:]`` operator, which returns a NumPy array.

In [14]:
temp = array[:]  # This will decompress the full array
type(temp)

numpy.ndarray


However it is often not necessary (or desirable) to load the whole array into memory. We can easily read just small parts of NDArray arrays to a NumPy array, quickly, via standard indexing routines.

In [15]:
res1 = array[0]  # get first element
res2 = array[6:10]  # get slice
print(f"Got one element (of shape {res1.shape}) and slice of shape {res2.shape}.")
print(res1)
print(res2)

Got one element (of shape (10000,)) and slice of shape (4, 10000).
[   0    1    2 ... 9997 9998 9999]
[[60000 60001 60002 ... 69997 69998 69999]
 [70000 70001 70002 ... 79997 79998 79999]
 [80000 80001 80002 ... 89997 89998 89999]
 [90000 90001 90002 ... 99997 99998 99999]]


We can modify the data in the array using standard NumPy indexing too, using either NumPy or NDArray arrays as the data source.  For example, we can set the first row to zeros (using an NDArray array) and the first column to ones (using a NumPy array)

In [16]:
array[0, :] = blosc2.zeros(10000, dtype=array.dtype)
array[:, 0] = np.ones(10000, dtype=array.dtype)
print(array)

Note that ``array`` is still an NDArray array. Let's check that the entries were correctly modified.

In [17]:
print(array[0, 0])
print(array[0, :])
print(array[:, 0])

1
[1 0 0 ... 0 0 0]
[1 1 1 ... 1 1 1]


## Enlarging the array
Existing arrays can be enlarged. This is one operation that is greatly enhanced by the chunking procedure implemented in NDArray arrays.

In [18]:
%time array.resize((10_001, 10_000))
print(array.shape)
array[10_000, :] = 1
array[10_000, :]

CPU times: user 18 μs, sys: 1e+03 ns, total: 19 μs
Wall time: 21 μs
(10001, 10000)


array([1, 1, 1, ..., 1, 1, 1], shape=(10000,), dtype=int32)

In [19]:
%time nparray2 = np.resize(nparray, (10_001, 10_001))

CPU times: user 56.9 ms, sys: 82.6 ms, total: 139 ms
Wall time: 139 ms


Enlarging a NumPy array requires a full copy of the data, since underlying data are stored contiguously in memory, which is very costly: new memory to hold the extended array is allocated, the old data is copied to part of the new memory, and then the new data is written to the remaining new memory.
Enlarging is a much faster operation for NDArray arrays because data is chunked, and the chunks may be stored non-contiguously in memory, so one may simply write the necessary new chunks to some arbitrary address in memory and leave the old chunks untouched. The references to the new chunk addresses are then added in the NDArray container, which is a very quick operation.

You can also shrink the array.

In [20]:
array.resize((9_000, 10_000))
print(array.shape)
print(array[8_999])  # This works
# array[9_000]  # This will raise an exception

(9000, 10000)
[       1 89990001 89990002 ... 89999997 89999998 89999999]


## Persistent data
We can use the `save()` method to store the array on disk.  This is very useful when you are working with a large array but do not need to access it often.


In [21]:
array.save("array_tutorial.b2nd", mode="w")  # , contiguous=True)
!ls -lh array_tutorial.b2nd

-rw-r--r--@ 1 faltet  staff   1,2M 10 des.  08:33 array_tutorial.b2nd



For arrays, it is usual to use the `.b2nd` extension. Now let's open the saved array and check that the data saved correctly (decompressing first to be able to compare):

In [22]:
array2 = blosc2.open("array_tutorial.b2nd")
np.array_equal(array2, array) # Make sure saved array matches original

True

By using the ``urlpath`` parameter, we can also write directly to disk using the other constructors we saw previously.

In [23]:
arr = blosc2.arange(np.prod(shape), shape=shape, dtype=np.int32, urlpath="arange.b2nd", mode="w")
print("3 first rows of arr:", arr[:3])
it = ((-x + 1, x - 2, rng.normal()) for x in range(N))
sa = blosc2.fromiter(it, dtype="i4,f4,f8", shape=(N,), urlpath="sa-1M.b2nd", mode="w")
print("3 first rows of sa:", sa[:3])
b2array = blosc2.asarray(nparray, urlpath="linspace.b2nd", mode="w")
print("3 first rows of b2array:", b2array[:3])

3 first rows of arr: [[    0     1     2 ...  9997  9998  9999]
 [10000 10001 10002 ... 19997 19998 19999]
 [20000 20001 20002 ... 29997 29998 29999]]
3 first rows of sa: [( 1, -2., -0.61881834) ( 0, -1., -0.21309748) (-1,  0., -0.39536403)]
3 first rows of b2array: [[0.00000000e+00 1.00000001e-06 2.00000002e-06 ... 9.99700010e-03
  9.99800010e-03 9.99900010e-03]
 [1.00000001e-02 1.00010001e-02 1.00020001e-02 ... 1.99970002e-02
  1.99980002e-02 1.99990002e-02]
 [2.00000002e-02 2.00010002e-02 2.00020002e-02 ... 2.99970003e-02
  2.99980003e-02 2.99990003e-02]]
